In [62]:
import pandas as pd
import numpy as np

In [63]:
FILEPATH = "../../../data/ml_ready/999_cleaned_ml.csv"

df = pd.read_csv(FILEPATH)

In [64]:
df.describe()

,Time,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,E56,E57,E58,E59,E60,E61,E62,E63,E64,E65
count,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,...,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000,140400.000000
mean,399.500000,0.326398,5.497308,4.391559,4.275709,1.146284,3.733715,7.418393,4.196285,3.497647,...,3.541667,5.317720,2.871461,5.557729,5.203823,2.860058,2.860058,2.860058,2.860058,2.860058
std,346.411275,86.360609,38.543504,20.696607,14.933305,49.129299,17.376537,83.996985,31.516075,17.203462,...,19.621094,18.686428,28.594721,26.990124,36.443826,14.427196,14.427196,14.427196,14.427196,14.427196
min,-200.000000,-1124.686768,-254.052292,-100.759499,-69.306091,-291.487335,-87.718178,-595.640442,-170.522079,-101.009491,...,-91.266334,-93.246368,-137.076279,-120.142029,-210.062347,-67.642334,-67.642334,-67.642334,-67.642334,-67.642334
25%,99.750000,-24.966866,-10.523659,-6.714929,-4.513655,-16.734442,-6.011759,-5.136086,-9.946945,-5.938697,...,-7.146462,-5.899804,-10.543979,-9.190728,-10.585828,-5.700868,-5.700868,-5.700868,-5.700868,-5.700868
50%,399.500000,0.778072,3.349213,3.181908,3.088097,2.859373,3.008024,2.748234,3.630306,2.909085,...,3.276697,4.216379,2.944930,5.093300,3.425295,2.218385,2.218385,2.218385,2.218385,2.218385
75%,699.250000,27.042214,19.910015,15.299859,12.325269,21.521421,12.990615,11.743638,18.687775,12.366368,...,14.254051,16.162963,17.129122,19.675857,20.405481,11.153640,11.153640,11.153640,11.153640,11.153640
max,999.000000,489.538086,195.419922,96.011970,89.002991,332.491638,89.900177,1772.739380,151.228745,84.758247,...,116.127716,83.077087,167.497192,160.136810,201.659683,80.635612,80.635612,80.635612,80.635612,80.635612


In [65]:
# samples are df rows mod 1200, pre is 0-199, post is 200-1199

# extract pre rows and post rows, randomly choose 200-400ms as post
pre = df[df.Time < 0]
post = df[df.Time >= 200][df.Time < 400]

c:\users\karl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [66]:
pre = pre.drop(columns=["Time", "E65"], axis=1)
post = post.drop(columns=["Time", "E65"], axis=1)

In [67]:
timeFrame = 200

preX = pre.values
(i,j) = preX.shape
preX = np.reshape(preX, (i // 200, j * 200))

In [68]:
postX = post.values
(i,j) = postX.shape
postX = np.reshape(postX, (i // 200, j * 200))

In [69]:
(preN, preD) = preX.shape
(postN, postD) = postX.shape

In [70]:
X = np.concatenate((preX, postX), axis=0)
(n, d) = X.shape
assert(n == preN + postN)
assert(d == postD)
assert(d == preD)

In [71]:
y = np.concatenate((np.zeros(preN), np.ones(postN)))
assert(y.shape == (n,))

In [72]:
rand_indices = np.random.permutation(n)
split = int(n * 0.8)
train_idx, valid_idx = rand_indices[:split], rand_indices[split:]
train_X, valid_X, train_y, valid_y = X[train_idx, :], X[valid_idx, :], y[train_idx], y[valid_idx]

In [78]:
from sklearn.svm import SVC
model = SVC(gamma='auto', kernel='linear')
model.fit(train_X, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [79]:
Xhat = model.predict(valid_X)

In [80]:
np.mean(Xhat != valid_y)

0.2765957446808511

In [82]:
Xhat

array([1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1.])

In [83]:
valid_y

array([1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.])

In [84]:
np.mean(model.predict(train_X) != train_y)

0.0